In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gradio==4.32.0
!pip3 install underthesea
import pandas as pd
import underthesea
import regex as re
from underthesea import word_tokenize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.8 MB/

In [3]:
# Cài đặt một số hàm tiền xử lý văn bản cần thiết
# !pip3 install --user underthesea
# import regex as re
# from underthesea import word_tokenize

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

In [4]:
file_path = '/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/FinalDataset.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

In [5]:
def text_preprocess(document):
    # xóa html code
    document = remove_html(document)
    # xóa xuống dòng
    document= document.replace('\n', ' ')
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [6]:
document = text_preprocess(data)

In [7]:
count = {}
for line in open('/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/FinalDataset.txt'):
  # print(f"====>{line}")
  key = line.split()[0]
  count[key] = count.get(key, 0) + 1

for key in count:
    print(key, count[key])

Am_nhac 100
Am_thuc 101
Bat_dong_san 100
Bong_da 101
Chung_khoan 101
Du_lich 101
Gia_dinh 101
Giao_duc 101
Khong_gian_song 101
Lam_dep 101
My_thuat 91
Thoi_trang 91


In [8]:
# Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 12
vocab = {}
label_vocab = {}
for line in open('/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/FinalDataset.txt'):
    words = line.split()
    # lưu ý từ đầu tiên là nhãn
    label = words[0]
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words[1:]:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

count = {}
for word in vocab:
    if len(vocab[word]) == total_label:
        count[word] = min([label_vocab[x][word] for x in label_vocab])

sorted_count = sorted(count, key=count.get, reverse=True)
for word in sorted_count[:100]:
    print(word, count[word])

và 298
có 250
cho 222
của 220
với 220
là 214
một 187
không 146
các 141
được 134
những 117
trong 108
để 98
như 88
trên 82
cũng 77
nhiều 77
người 73
khi 72
từ 69
ra 68
sẽ 68
vào 66
này 66
lại 64
đã 60
rất 58
đến 57
chỉ 56
thể 56
ở 56
làm 54
phải 53
- 52
nhưng 51
thành 50
nên 45
mà 44
còn 43
bị 40
đó 36
về 32
thì 32
hơn 32
theo 32
hợp 31
bằng 31
gia 30
hiện 29
hai 29
cách 29
lên 29
đầu 28
cả 26
tiếp 25
phần 24
trở 24
cùng 24
đi 23
nước 23
gian 22
hình 22
thực 22
đều 22
nhà 22
vì 21
mới 21
nhất 21
vừa 21
công 20
đây 20
đồng 20
khác 19
đường 19
cần 19
hay 19
nào 19
ngày 19
qua 18
chính 18
quá 18
thường 18
sau 17
số 17
điều 17
thêm 17
thế 16
thời 16
thấy 16
tại 15
mặt 15
đủ 14
chọn 14
ngoài 14
việc 13
Một 13
do 13
hàng 13
từng 13
cầu 13


In [9]:
# loại stopword khỏi dữ liệu
# lưu file dùng về sau
stopword = set()
with open('stopwords.txt', 'w') as fp:
    for word in sorted_count[:100]:
        stopword.add(word)
        fp.write(word + '\n')

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)


with open('dataset.prep', 'w') as fp:
    for line in open('/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/FinalDataset.txt'):
        line = remove_stopwords(line)
        fp.write(line + '\n')

In [10]:
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
test_percent = 0.3

text = []
label = []

for line in open('dataset.prep'):
    words = line.strip().split()
    label.append(words[0])
    text.append(' '.join(words[1:]))

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size=test_percent, random_state=42)

# Lưu train/test data
# Giữ nguyên train/test để về sau so sánh các mô hình cho công bằng
with open('train.txt', 'w') as fp:
    for x, y in zip(X_train, y_train):
        fp.write('{} {}\n'.format(y, x))

with open('test.txt', 'w') as fp:
    for x, y in zip(X_test, y_test):
        fp.write('{} {}\n'.format(y, x))

# encode label
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print(list(label_encoder.classes_), '\n')
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)


['Am_nhac', 'Am_thuc', 'Bat_dong_san', 'Bong_da', 'Chung_khoan', 'Du_lich', 'Gia_dinh', 'Giao_duc', 'Khong_gian_song', 'Lam_dep', 'My_thuat', 'Thoi_trang'] 



In [11]:
MODEL_PATH = '/content/drive/MyDrive/AI - Trí tuệ nhân tạo/FINAL ASSIGNMENT/ML_modal'

import os
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

In [13]:
import pickle
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
# Khởi tạo pipeline
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())
                    ])
# Đánh giá mô hình bằng phương pháp cross-validation
scores = cross_val_score(text_clf, X_train, y_train, cv=4)
train_time = time.time() - start_time
print(f'Mean cross-validation accuracy: {np.mean(scores)}')
# Huấn luyện mô hình trên toàn bộ tập huấn luyện
text_clf = text_clf.fit(X_train, y_train)
# Lưu mô hình đã huấn luyện
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "NaiveBayes.pkl"), 'wb'))
# Tải lại mô hình và dự đoán trên tập kiểm tra
model = pickle.load(open(os.path.join(MODEL_PATH, "NaiveBayes.pkl"), 'rb'))
y_pred = model.predict(X_test)
# Đánh giá mô hình trên tập kiểm tra
print('Naive Bayes, Accuracy =', np.mean(y_pred == y_test))
# In báo cáo phân loại
nb_model = pickle.load(open(os.path.join(MODEL_PATH,"NaiveBayes.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print("________________________________________________________________________________")
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

NameError: name 'start_time' is not defined

In [ ]:
import os
import time
import pickle
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report




from sklearn.preprocessing import LabelEncoder
# Khởi tạo pipeline
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,1),
                                             max_df=0.8,
                                             max_features=None)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC(gamma='scale'))
                    ])
# Đánh giá mô hình bằng phương pháp cross-validation
scores = cross_val_score(text_clf, X_train, y_train, cv=4)
print(f'Mean cross-validation accuracy: {np.mean(scores)}')
# Huấn luyện mô hình trên toàn bộ tập huấn luyện
text_clf = text_clf.fit(X_train, y_train)
# Lưu mô hình đã huấn luyện
pickle.dump(text_clf, open(os.path.join(MODEL_PATH, "svm.pkl"), 'wb'))
# Tải lại mô hình và dự đoán trên tập kiểm tra
model = pickle.load(open(os.path.join(MODEL_PATH, "svm.pkl"), 'rb'))
y_pred = model.predict(X_test)
# Đánh giá mô hình trên tập kiểm tra
print('SVM, Accuracy =', np.mean(y_pred == y_test))
# In báo cáo phân loại
nb_model = pickle.load(open(os.path.join(MODEL_PATH,"svm.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

In [ ]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report
nb_model = pickle.load(open(os.path.join(MODEL_PATH,"NaiveBayes.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

In [ ]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join(MODEL_PATH,"LogisticRegression.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

In [ ]:
# Xem kết quả trên từng nhãn
from sklearn.metrics import classification_report

nb_model = pickle.load(open(os.path.join(MODEL_PATH,"svm.pkl"), 'rb'))
y_pred = nb_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(label_encoder.classes_)))

In [14]:

document = "Chứng khoán những ngày gần đây thực sự là trao đảo. Ba phần tư cổ phiếu trên thị trường giảm giá, lực bán dâng cao liên tục đẩy thanh khoản vượt 35.500 tỷ đồng trong phiên VN-Index sụt hơn 19 điểm.\
Dù tăng vượt 1.280 điểm vào hôm qua, VN-Index bị nhuộm đỏ ngay những phút đầu phiên giao dịch hôm nay, mất gần 8 điểm. Sau khoảng một tiếng, chỉ số này mới cải thiện lên tham chiếu, giằng co một lúc rồi lấy lại sắc xanh nhờ lực cầu tiếp ứng. Tuy nhiên áp lực chốt lời ở nhóm bluechip vẫn rất cao khiến chỉ số nhanh chóng rơi về dưới tham chiếu trước khi nghỉ trưa.\
Sang buổi chiều, chỉ số đại diện sàn HoSE khoác sắc đỏ liên tục. Nửa đầu buổi, chỉ số này chủ yếu giằng co quanh 1.270 điểm.\
Đến sau 14h, xu hướng bán tháo lan nhanh từ nhóm bluechip sang hàng loạt cổ phiếu. Chỉ số bất ngờ giảm một mạch về sát 1.250 điểm, tức thấp hơn tham chiếu khoảng 30 điểm. Tuy nhiên, thị trường nhanh chóng cải thiện trước khi bước vào phiên ATC.\
VN-Index đóng cửa ở gần 1.262 điểm, giảm hơn 19 điểm so với hôm trước. Đây là phiên điều chỉnh mạnh nhất hơn một tháng qua"

document = text_preprocess(document)
document = remove_stopwords(document)
svm_model = pickle.load(open(os.path.join(MODEL_PATH,"NaiveBayes.pkl"), 'rb'))
label = svm_model.predict([document])
print('Predict label:', label_encoder.inverse_transform(label))

Predict label: ['Chung_khoan']


In [15]:
import gradio as gr

def predict_label(content):
  content = text_preprocess(content)
  content = remove_stopwords(content)
  svm_model = pickle.load(open(os.path.join(MODEL_PATH,"NaiveBayes.pkl"), 'rb'))
  label = svm_model.predict([content])
  return "Bài báo thuộc chủ đề: " + str(label_encoder.inverse_transform(label))

demo = gr.Interface(fn=predict_label, inputs="text", outputs="text")
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://97e74f8d047bf7ee16.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
